In [262]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

RSEED = 42

In [263]:
from sklearn.pipeline import Pipeline # focus on this one
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report 

In [264]:
df = pd.read_pickle("data/df_train_1.pkl")
df

,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,...,L3_NO2_cloud_fraction,L3_CO_CO_column_number_density,L3_CO_cloud_height,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_cloud_fraction,L3_O3_O3_column_number_density,L3_O3_cloud_fraction,L3_SO2_SO2_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction
0,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,0.00804,...,0.006507,0.021080,267.017184,0.000064,0.000000,0.119095,0.000000,-0.000127,-1.861476,0.000000
1,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,0.00839,...,0.018360,0.022017,61.216687,0.000171,0.059433,0.115179,0.059433,0.000150,-1.452612,0.059433
2,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,0.00750,...,0.015904,0.020677,134.700335,0.000124,0.082063,0.115876,0.082063,0.000150,-1.572950,0.082063
3,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,0.00391,...,0.055765,0.021207,474.821444,0.000081,0.121261,0.141557,0.121261,0.000227,-1.239317,0.121261
4,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,0.00535,...,0.028530,0.037766,926.926310,0.000140,0.037919,0.126369,0.037919,0.000390,0.202489,0.037919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30552,2020-03-15,YWSFY6Q,22.0,14.0,83.0,3848.86,72,6.700000,68.300003,0.00352,...,0.001107,0.039941,192.388239,0.000024,0.001310,0.174995,0.001310,0.000312,-1.953480,0.001310
30553,2020-03-16,YWSFY6Q,53.0,30.0,146.0,9823.87,72,6.300000,77.700005,0.00341,...,0.004726,0.037872,61.379434,-0.000014,0.007644,0.157659,0.007644,0.000362,-2.178236,0.007644
30554,2020-03-17,YWSFY6Q,85.0,52.0,153.0,8900.85,72,7.100000,68.500000,0.00356,...,0.026249,0.038539,1572.596434,0.000094,0.025447,0.168295,0.025447,0.000107,-2.365827,0.025447
30555,2020-03-18,YWSFY6Q,103.0,33.0,149.0,13963.90,72,19.100000,66.300003,0.00523,...,0.144318,0.038757,846.961465,0.000063,0.103292,0.160637,0.173391,0.000014,-2.784346,0.153445


In [265]:
# create windstrength column
#df['windstrength'] = np.sqrt(df.u_component_of_wind_10m_above_ground**2 + df.v_component_of_wind_10m_above_ground**2)
#df.windstrength.describe()

In [266]:
df.columns

Index(['Date', 'Place_ID', 'target', 'target_min', 'target_max',
       'target_variance', 'target_count',
       'precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_AER_AI_absorbing_aerosol_index', 'L3_CLOUD_cloud_base_height',
       'L3_CLOUD_cloud_fraction', 'L3_CLOUD_cloud_optical_depth',
       'L3_NO2_NO2_column_number_density', 'L3_NO2_absorbing_aerosol_index',
       'L3_NO2_cloud_fraction', 'L3_CO_CO_column_number_density',
       'L3_CO_cloud_height', 'L3_HCHO_tropospheric_HCHO_column_number_density',
       'L3_HCHO_cloud_fraction', 'L3_O3_O3_column_number_density',
       'L3_O3_cloud_fraction', 'L3_SO2_SO2_column_number_density',
       'L3_SO2_absorbing_aerosol_index', 'L3_SO2_cloud_fraction'],
      dtype='object')

In [267]:
# Dropping the unnecessary columns 
df.drop(['Date',
         #'Place_ID',
         'target_min',
         'target_max',
         'target_variance',
         'target_count',
         #'u_component_of_wind_10m_above_ground',
         #'v_component_of_wind_10m_above_ground'
         ], axis=1, inplace=True)
df.columns
df.columns

Index(['Place_ID', 'target', 'precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_AER_AI_absorbing_aerosol_index', 'L3_CLOUD_cloud_base_height',
       'L3_CLOUD_cloud_fraction', 'L3_CLOUD_cloud_optical_depth',
       'L3_NO2_NO2_column_number_density', 'L3_NO2_absorbing_aerosol_index',
       'L3_NO2_cloud_fraction', 'L3_CO_CO_column_number_density',
       'L3_CO_cloud_height', 'L3_HCHO_tropospheric_HCHO_column_number_density',
       'L3_HCHO_cloud_fraction', 'L3_O3_O3_column_number_density',
       'L3_O3_cloud_fraction', 'L3_SO2_SO2_column_number_density',
       'L3_SO2_absorbing_aerosol_index', 'L3_SO2_cloud_fraction'],
      dtype='object')

#### Categorical and numerical variables

In [268]:
# Creating list for categorical predictors/features 
# (dates are also objects so if you have them in your data you would deal with them first)
cat_features = list(df.columns[df.dtypes==object])
cat_features

['Place_ID']

In [269]:
# Creating list for numerical predictors/features
# Since 'target' is our target variable we will exclude this feature from this list of numerical predictors 
num_features = [#'target_min', 'target_max', 'target_variance', 'target_count',
       'precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground',
       'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground'
       ]
num_features

['precipitable_water_entire_atmosphere',
 'relative_humidity_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'temperature_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground']

In [270]:
# Creating list for numerical predictors/features
# Since 'outcome' is our target variable we will exclude this feature from this list of numerical predictors 
miss_features = ['L3_AER_AI_absorbing_aerosol_index', 
       'L3_CLOUD_cloud_base_height',
       'L3_CLOUD_cloud_fraction', 
       'L3_CLOUD_cloud_optical_depth',
       'L3_NO2_NO2_column_number_density',
       'L3_NO2_cloud_fraction', 
       'L3_CO_cloud_height',
       'L3_HCHO_tropospheric_HCHO_column_number_density',
       'L3_HCHO_cloud_fraction', 
       'L3_SO2_SO2_column_number_density',
       'L3_SO2_cloud_fraction']
miss_features

['L3_AER_AI_absorbing_aerosol_index',
 'L3_CLOUD_cloud_base_height',
 'L3_CLOUD_cloud_fraction',
 'L3_CLOUD_cloud_optical_depth',
 'L3_NO2_NO2_column_number_density',
 'L3_NO2_cloud_fraction',
 'L3_CO_cloud_height',
 'L3_HCHO_tropospheric_HCHO_column_number_density',
 'L3_HCHO_cloud_fraction',
 'L3_SO2_SO2_column_number_density',
 'L3_SO2_cloud_fraction']

In [271]:
replace_features = ["L3_NO2_absorbing_aerosol_index",
                    "L3_CO_CO_column_number_density",
                    "L3_O3_O3_column_number_density",
                    "L3_O3_cloud_fraction",
                    "L3_SO2_absorbing_aerosol_index"]
replace_features

['L3_NO2_absorbing_aerosol_index',
 'L3_CO_CO_column_number_density',
 'L3_O3_O3_column_number_density',
 'L3_O3_cloud_fraction',
 'L3_SO2_absorbing_aerosol_index']

#### Train-Test-Split

In [272]:
# Define predictors and target variable
X = df.drop('target', axis=1)
y = df['target']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

We have 30557 observations in our dataset and 23 features
Our target vector has also 30557 values


In [273]:
# Split into train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED) 

In [274]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (21389, 23)
X_test shape: (9168, 23)
y_train shape: (21389,)
y_test shape: (9168,)


#### Preprocessing Pipeline

In [275]:
#from sklearn.pipeline import Pipeline

# Pipline for numerical features
# Initiating Pipeline and calling one step after another
# each step is built as a list of (key, value)
# key is the name of the processing step
# value is an estimator object (processing step)
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])
# Pipeline for missing values: in this case missing values are nan!
miss_pipeline = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median',missing_values=np.nan)),
    ('std_scaler', StandardScaler())
])
# replace 0 values
# Pipeline for missing values: in this case missing values are 0!
replace_pipeline = Pipeline([
    ('imputer_null', SimpleImputer(strategy='constant', fill_value = 0)),
    ('imputer_nan', SimpleImputer(strategy='median', missing_values= 0)),
    ('std_scaler', StandardScaler())
])

# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant')),
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [276]:
#from sklearn.compose import ColumnTransformer

# Complete pipeline for numerical and categorical features
# 'ColumnTranformer' applies transformers (num_pipeline/ cat_pipeline)
# to specific columns of an array or DataFrame (num_features/cat_features)
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
    ('miss', miss_pipeline, miss_features),
    ('replace', replace_pipeline, replace_features)
])


### Predictive modeling using Pipelines and GridSearch

#### Decision Tree

In [277]:
# Building a full pipeline with our preprocessor and a DecisionTreeRegressor
pipe_destree = Pipeline([
    ('preprocessor', preprocessor),
    ('destree', DecisionTreeRegressor())
])

In [278]:
# Making predictions on the training set using cross validation as well as calculating the probabilities
# cross_val_predict expects an estimator (model), X, y and nr of cv-splits (cv)
y_train_predicted = cross_val_predict(pipe_destree, X_train, y_train, cv=5)

In [279]:
# Mean Squared Error
print('MSE Dtree Train:\n', mean_squared_error(y_train, y_train_predicted))

# Root Mean Squared Error
print('RMSE Dtree Train:\n', mean_squared_error(y_train, y_train_predicted, squared = False))

# R^2 Score
print('R^2 Dtree Train:\n', r2_score(y_train, y_train_predicted))


MSE Dtree Train:
 1906.8871667679648
RMSE Dtree Train:
 43.66791919439218
R^2 Dtree Train:
 0.1376827942524198


#### Optimizing via GridSearch

In [280]:
# Defining parameter space for grid-search. Since we want to access the classifier step (called 'logreg') in our pipeline 
# we have to add 'logreg__' infront of the corresponding hyperparameters. 
param_destree = { "destree__max_depth" : [100000000]}
grid_destree = GridSearchCV(pipe_destree, param_grid=param_destree, scoring = 'r2', cv=5,
                           verbose=1, n_jobs=-1)

In [281]:
grid_destree.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['precipitable_water_entire_atmosphere',
                                                                          'relative_humidity_2m_above_ground',
                                                                          'specific_humidity_2m_above_ground',
                                                                          'temperature_2m_above_ground',
                                                                          'u_component_of_wind_10m_above_ground',
                                                                          '

In [282]:
# Show best parameters
print('Best score:\n{:.2f}'.format(grid_destree.best_score_))
print("Best parameters:\n{}".format(grid_destree.best_params_))

Best score:
0.16
Best parameters:
{'destree__max_depth': 100000000}


In [283]:
# Save best model (including fitted preprocessing steps) as best_model 
best_model = grid_destree.best_estimator_
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['precipitable_water_entire_atmosphere',
                                                   'relative_humidity_2m_above_ground',
                                                   'specific_humidity_2m_above_ground',
                                                   'temperature_2m_above_ground',
                                                   'u_component_of_wind_10m_above_ground',
                                                   'v_component_of_wind_10m_above_ground']),
                                                 ('cat',...
                                                                   SimpleImputer(fill_value=0,
                                                     

In [284]:
# Calculating the accuracy, recall and precision for the test set with the optimized model
y_test_predicted = best_model.predict(X_test)

# Mean Squared Error
print('MSE Dtree Test:\n', mean_squared_error(y_test, y_test_predicted))

# Root Mean Squared Error
print('RMSE Dtree Test:\n', mean_squared_error(y_test, y_test_predicted, squared = False))

# R^2 Score
print('R^2 Dtree Test:\n', r2_score(y_test, y_test_predicted))

MSE Dtree Test:
 1776.2950632635254
RMSE Dtree Test:
 42.14611563671705
R^2 Dtree Test:
 0.17753983637421367
